# 1. Importing Packages and Collecting Data


In [1]:
import pandas as pd
import numpy as np
from scipy import stats

'''Customize visualization
Seaborn and matplotlib visualization.'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

'''Plotly visualization .'''
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected = True) # Required to use plotly offline in jupyter notebook

'''Display markdown formatted output like bold, italic bold etc.'''
from IPython.display import Markdown
def bold(string):
    display(Markdown(string))

In [2]:
'''Read in export and import data from CSV file'''
df_train = pd.read_csv('../input/bigquery-geotab-intersection-congestion/train.csv')
df_test = pd.read_csv('../input/bigquery-geotab-intersection-congestion/test.csv')

# 4. Feature Engineering


## <span style='color:darkgreen;background:yellow'>4.1. Intersection ID 

Making a new columns of IntersectionId with city name.

In [3]:
df_train['Intersection'] = df_train['IntersectionId'].astype(str) + df_train['City']
df_test['Intersection'] = df_test['IntersectionId'].astype(str) + df_test['City']
print(df_train['Intersection'].sample(6).values)

In [4]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
le.fit(pd.concat([df_train['Intersection'], df_test['Intersection']]).drop_duplicates().values)
df_train['Intersection'] = le.transform(df_train['Intersection'])
df_test['Intersection'] = le.transform(df_test['Intersection'])
print(df_train['Intersection'].sample(6).values)

## <span style='color:darkgreen;background:yellow'>4.2. Encoding Street Names 
We are encode the street name according to its road type. 

In [6]:
# Reference: https://www.kaggle.com/bgmello/how-one-percentile-affect-the-others

'''Let's use the following road types: Street, Avenue, Road, Boulevard, Broad and Drive'''
road_encoding = {
    'Road': 1,
    'Street': 2,
    'Avenue': 2,
    'Drive': 3,
    'Broad': 3,
    'Boulevard': 4
}

In [7]:
def encode(x):
    if pd.isna(x):
        return 0
    for road in road_encoding.keys():
        if road in x:
            return road_encoding[road]
        
    return 0

In [8]:
df_train['EntryTypeStreet'] = df_train['EntryStreetName'].apply(encode)
df_train['ExitTypeStreet'] = df_train['ExitStreetName'].apply(encode)
df_test['EntryTypeStreet'] = df_test['EntryStreetName'].apply(encode)
df_test['ExitTypeStreent'] = df_test['ExitStreetName'].apply(encode)
print(df_train['EntryTypeStreet'].sample(10).values)

In [9]:
df_train["same_street_exact"] = (df_train["EntryStreetName"] ==  df_train["ExitStreetName"]).astype(int)
df_test["same_street_exact"] = (df_test["EntryStreetName"] ==  df_test["ExitStreetName"]).astype(int)

## <span style='color:darkgreen;background:yellow'>4.3. Encoding Cordinal Direction 
Turn Direction: 

The cardinal directions can be expressed using the equation: $$ \frac{\theta}{\pi} $$

Where $\theta$ is the angle between the direction we want to encode and the north compass direction, measured clockwise.


Reference: 
* https://www.kaggle.com/danofer/baseline-feature-engineering-geotab-69-5-lb
* This is an important feature, as shown by janlauge here : https://www.kaggle.com/janlauge/intersection-congestion-eda

* We can fill in this code in python (e.g. based on: https://www.analytics-link.com/single-post/2018/08/21/Calculating-the-compass-direction-between-two-points-in-Python , https://rosettacode.org/wiki/Angle_difference_between_two_bearings#Python , https://gist.github.com/RobertSudwarts/acf8df23a16afdb5837f )

In [10]:
'''Defineing the directions'''
directions = {
    'N': 0,
    'NE': 1/4,
    'E': 1/2,
    'SE': 3/4,
    'S': 1,
    'SW': 5/4,
    'W': 3/2,
    'NW': 7/4
}

In [11]:
df_train['EntryHeading'] = df_train['EntryHeading'].map(directions)
df_train['ExitHeading'] = df_train['ExitHeading'].map(directions)

df_test['EntryHeading'] = df_test['EntryHeading'].map(directions)
df_test['ExitHeading'] = df_test['ExitHeading'].map(directions)

df_train['diffHeading'] = df_train['EntryHeading']- df_train['ExitHeading']  
df_test['diffHeading'] = df_test['EntryHeading']- df_test['ExitHeading']

display(df_train[['ExitHeading','EntryHeading','diffHeading']].drop_duplicates().head(5))

## <span style='color:darkgreen;background:yellow'>4.5 standardizing of lat-long

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
lat_long = ['Latitude', 'Longitude']
for col in lat_long:
    df_train[col] = (scaler.fit_transform(df_train[col].values.reshape(-1, 1)))
    df_test[col] = (scaler.fit_transform(df_test[col].values.reshape(-1, 1)))

## <span style='color:darkgreen;background:yellow'>4.6 Droping the variables 

In [14]:
"""Let's see the columns of data"""
df_train.columns.values

In [16]:
"""Let's drop the unwanted variables from test and train dataset"""
df_train.drop(['RowId', 'IntersectionId', 'EntryStreetName', 'ExitStreetName', 'Path', 'City'], axis=1, inplace=True)
df_test.drop(['RowId', 'IntersectionId', 'EntryStreetName', 'ExitStreetName', 'Path', 'City'], axis=1, inplace=True)

# 5. Seting X and Y

In [18]:
'''Function to reduce the DF size'''
# source: https://www.kaggle.com/kernels/scriptcontent/3684066/download

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [19]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

In [27]:
'''Seting X and Y'''
target_var = df_train.iloc[:, 7:22]
X_train = df_train.drop(target_var,axis = 1)

y1_train = df_train["TotalTimeStopped_p20"]
y2_train = df_train["TotalTimeStopped_p50"]
y3_train = df_train["TotalTimeStopped_p80"]
y4_train = df_train["DistanceToFirstStop_p20"]
y5_train = df_train["DistanceToFirstStop_p50"]
y6_train = df_train["DistanceToFirstStop_p80"]

X_test = df_test

In [28]:
"""Let's have a final look at our data"""
bold('**Data Dimension for Model Building:**')
print('Input matrix dimension:', X_train.shape)
print('Output vector dimension:',y1_train.shape)
print('Test data dimension:', X_test.shape)

# 6. Model Building & Evaluation

In [30]:
X_train.describe()

In [41]:
"""pecifying categorical features"""
cat_feat = ['Hour', 'Weekend','Month', 'same_street_exact', 'Intersection', 'EntryTypeStreet', 'ExitTypeStreet']

In [42]:
all_preds ={0:[],1:[],2:[],3:[],4:[],5:[]}
all_target = [y1_train, y2_train, y3_train, y4_train, y5_train, y6_train]

In [43]:
# Reference: 
# https://medium.com/analytics-vidhya/hyperparameters-optimization-for-lightgbm-catboost-and-xgboost-regressors-using-bayesian-6e7c495947a9
# https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html#for-faster-speed

'''Importing Libraries'''
import lightgbm as lgb
from bayes_opt import BayesianOptimization

dtrain = lgb.Dataset(data=X_train, label=y1_train)

'''Define objective function'''
def hyp_lgbm(num_leaves, feature_fraction, bagging_fraction, max_depth, min_split_gain, min_child_weight, lambda_l1, lambda_l2):
      
        params = {'application':'regression','num_iterations': 10,
                  'learning_rate':0.01,
                  'metric':'rmse'} # Default parameters
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        
        cv_results = lgb.cv(params, dtrain, nfold=5, seed=44, categorical_feature=cat_feat, stratified=False,
                            verbose_eval =None)
#         print(cv_results)
        return -np.min(cv_results['rmse-mean'])

In [44]:
''' Define search space of hyperparameters'''
pds = {'num_leaves': (100, 230),
          'feature_fraction': (0.1, 0.5),
          'bagging_fraction': (0.8, 1),
          'lambda_l1': (0,3),
          'lambda_l2': (0,5),
          'max_depth': (8, 19),
          'min_split_gain': (0.001, 0.1),
          'min_child_weight': (1, 20)
        }

In [47]:
'''Define a surrogate model of the objective function and call it.'''
optimizer = BayesianOptimization(hyp_lgbm,pds,random_state=44)
                                  
# Optimize
optimizer.maximize(init_points=5, n_iter=12)

## 6.1 Retrain and Predict Using Optimized Hyperparameters

In [48]:
'''Best parameters after optimization'''
optimizer.max

In [49]:
p = optimizer.max['params']
param = {'num_leaves': int(round(p['num_leaves'])),
         'feature_fraction': p['feature_fraction'],
         'bagging_fraction': p['bagging_fraction'],
         'max_depth': int(round(p['max_depth'])),
         'lambda_l1': p['lambda_l1'],
         'lambda_l2':p['lambda_l2'],
         'min_split_gain': p['min_split_gain'],
         'min_child_weight': p['min_child_weight'],
         'learing_rate':0.05,
         'objective': 'regression',
         'boosting_type': 'gbdt',
         'verbose': 1,
         'seed': 44,
         'metric': 'rmse'
        }
param

In [ ]:
'''Instantiate the models with optimized hyperparameters.'''
train = X_train 
test = X_test 
from sklearn.model_selection import train_test_split

for i in range(len(all_preds)):
    print('Training and predicting for target {}'.format(i+1))
    X_train,X_test,y_train,y_test=train_test_split(train,all_target[i], test_size=0.2, random_state=31)
    xg_train = lgb.Dataset(X_train,
                           label = y_train
                           )
    xg_valid = lgb.Dataset(X_test,
                           label = y_test
                           )
    clf = lgb.train(param, xg_train, 10000, valid_sets = [xg_valid],categorical_feature=cat_feat,
                         verbose_eval=100, early_stopping_rounds = 200)
    all_preds[i] = clf.predict(test, num_iteration=clf.best_iteration)

In [ ]:
submission = pd.read_csv('../input/bigquery-geotab-intersection-congestion/sample_submission.csv')
#submission.head()

In [ ]:
dt = pd.DataFrame(all_preds).stack()
dt = pd.DataFrame(dt)
submission['Target'] = dt[0].values

In [ ]:
submission.head()

In [ ]:
submission.to_csv('lgbm2_submission.csv', index=False)